In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install PyDrive

In [ ]:
#@title Clone repo
!git clone https://github.com/xontoloyoo/InferTest
%cd InferTest

In [ ]:
#@title WAJIB
!pip install -q -r requirements.txt
!sudo apt update
!sudo apt install sox
!python src/download_models.py
!pip install gTTS

In [ ]:
#@title SALIN MODEL
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from shutil import copyfile
import shutil
import os

# Nama model yang dimasukkan oleh pengguna
nama_model = input("Masukkan nama model: ")

# Path ke folder sumber
source_folder = f'/content/drive/MyDrive/RVC/{nama_model}'

# Path ke folder tujuan di Google Drive
destination_folder = f'/content/InferTest/rvc_models/{nama_model}'

# Buat direktori baru di Colab sesuai dengan nama model
os.makedirs(destination_folder, exist_ok=True)

# Mencari semua file yang dimulai dengan "added_" di dalam folder sumber
matching_files = [file for file in os.listdir(source_folder) if file.startswith("added_") or file.startswith("total_")]

# Menyalin semua file yang sesuai dengan pola ke folder tujuan
for file in matching_files:
    source_file_path = os.path.join(source_folder, file)
    destination_file_path = os.path.join(destination_folder, file)
    shutil.copyfile(source_file_path, destination_file_path)
# Verifikasi bahwa file-file telah disalin dengan sukses
os.listdir(destination_folder)
folder = f'/content/drive/MyDrive/RVC/'

# Daftar file yang ingin Anda salin
files_to_copy = [
    f'weights/{nama_model}.pth'
]

for file_path in files_to_copy:
    folder = os.path.join(folder, file_path)
    destination_folder = os.path.join(destination_folder, os.path.basename(file_path))
    copyfile(folder, destination_folder)
    print(f"File '{folder}' telah berhasil disalin ke '{destination_folder}'")
    print("Semua file telah berhasil disalin")

In [ ]:
#@title Model Download Function

import os
import zipfile
import shutil
import urllib.request

BASE_DIR = os.getcwd()
rvc_models_dir = os.path.join(BASE_DIR, 'rvc_models')

def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)

    index_filepath, model_filepath = None, None
    for root, dirs, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)

            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)

    if not model_filepath:
        raise Exception(f'No .pth model file was found in the extracted zip. Please check {extraction_folder}.')

    # move model and index file to extraction folder
    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))

    # remove any unnecessary nested folders
    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

def download_online_model(url, dir_name):
    try:
        print(f'[~] Downloading voice model with name {dir_name}...')
        zip_name = url.split('/')[-1]
        extraction_folder = os.path.join(rvc_models_dir, dir_name)
        if os.path.exists(extraction_folder):
            raise Exception(f'Voice model directory {dir_name} already exists! Choose a different name for your voice model.')

        if 'pixeldrain.com' in url:
            url = f'https://pixeldrain.com/api/file/{zip_name}'

        urllib.request.urlretrieve(url, zip_name)

        print('[~] Extracting zip...')
        extract_zip(extraction_folder, zip_name)
        print(f'[+] {dir_name} Model successfully downloaded!')

    except Exception as e:
        raise Exception(str(e))

url = "https://pixeldrain.com/u/3tJmABXA" # @param {type:"string"}
dir_name = "Gura" # @param {type:"string"}

download_online_model(url, dir_name)

In [ ]:
# @title INFER V1
# @markdown Main Option | You also can input audio path inside "SONG_INPUT"
SONG_INPUT = "/content/drive/MyDrive/Angels_Like_You.wav" # @param {type:"string"}
RVC_DIRNAME = "FreyaNew" # @param {type:"string"}
PITCH_CHANGE = -1 # @param {type:"integer"}
PITCH_CHANGE_ALL = 0 # @param {type:"integer"}
# @markdown Voice Conversion Options
INDEX_RATE = 0.7 # @param {type:"number"}
FILTER_RADIUS = 3 # @param {type:"integer"}
PITCH_DETECTION_ALGO = "rmvpe" # @param ["rmvpe", "mangio-crepe"]
CREPE_HOP_LENGTH = 128 # @param {type:"integer"}
PROTECT = 0.33 # @param {type:"number"}
REMIX_MIX_RATE = 0.0  # @param {type:"number"}
# @markdown Output Format
OUTPUT_FORMAT = "wav" # @param ["mp3", "wav"]

# Menjalankan perintah dengan argumen yang ditentukan
#!python3 src/tes.py -i "{SONG_INPUT}" -dir "{RVC_DIRNAME}" -p {PITCH_CHANGE} -ir {INDEX_RATE} -fr {FILTER_RADIUS} -palgo "{PITCH_DETECTION_ALGO}" -hop {CREPE_HOP_LENGTH} -pro {PROTECT} -oformat "{OUTPUT_FORMAT}"
!python3 src/main.py -i "{SONG_INPUT}" -dir "{RVC_DIRNAME}" -p {PITCH_CHANGE} -ir {INDEX_RATE} -fr {FILTER_RADIUS} -palgo "{PITCH_DETECTION_ALGO}" -hop {CREPE_HOP_LENGTH} -pro {PROTECT} -oformat "{OUTPUT_FORMAT}"

In [ ]:
# @title PLAY AUDIO
from gtts import gTTS
from IPython.display import Audio

# Ganti "nama_file_audio.wav" dengan nama file audio yang Anda unggah
nama_file_audio = "/content/InferTest/song_output/6c831c6f926/Angels_Like_You (Versi FreyaNew).wav" # @param {type:"string"}

# Memutar file audio
Audio(nama_file_audio)




---


# ***SAMPAH TAPI GUNA***
---




In [ ]:
# @title INFER V2
# @markdown Main Option | You also can input audio path inside "SONG_INPUT"
SONG_INPUT = "/content/drive/MyDrive/Angels_Like_You.wav" # @param {type:"string"}
RVC_DIRNAME = "FreyaNew" # @param {type:"string"}
PITCH_CHANGE = 0.0 # @param {type:"number"}
PITCH_CHANGE_ALL = 0 # @param {type:"integer"}
# @markdown Voice Conversion Options
INDEX_RATE = 0.5 # @param {type:"number"}
FILTER_RADIUS = 3 # @param {type:"integer"}
PITCH_DETECTION_ALGO = "rmvpe" # @param ["rmvpe", "mangio-crepe"]
CREPE_HOP_LENGTH = 128 # @param {type:"integer"}
PROTECT = 0.33 # @param {type:"number"}
REMIX_MIX_RATE = 0.25  # @param {type:"number"}
# @markdown Output Format
OUTPUT_FORMAT = "mp3" # @param ["mp3", "wav"]

import subprocess

command = [
    "python",
    "src/main.py",
    "-i", SONG_INPUT,
    "-dir", RVC_DIRNAME,
    "-p", str(PITCH_CHANGE),
    "-k",
    "-ir", str(INDEX_RATE),
    "-fr", str(FILTER_RADIUS),
    "-rms", str(REMIX_MIX_RATE),
    "-palgo", PITCH_DETECTION_ALGO,
    "-hop", str(CREPE_HOP_LENGTH),
    "-pro", str(PROTECT),
    "-oformat", OUTPUT_FORMAT
]

# Open a subprocess and capture its output
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

# Print the output in real-time
for line in process.stdout:
    print(line, end='')

# Wait for the process to finish
process.wait()

In [ ]:
# @title HAPUS FOLDER
import shutil

folder_path = "/content/InferTest"  # @param {type:"string"}
shutil.rmtree(folder_path)
